<a href="https://colab.research.google.com/github/pardodan/project_A/blob/main/dataload_gad_dan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dataset loader**
this are the attemps of the data loader to load and preper the data before running it in the train model

there are 2 main challenges in this script:

1) load the data with the proper labels from the server 

2) crop the pictures to a "specific" size before running it it the model

#######TRY_1########

In [ ]:
##----------- imports ------------##

# torch
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torchvision
from torch import nn
from torchvision import datasets, transforms

# other
import os
import numpy as np
from urllib import request, error
from time import time
import random
from PIL import Image
from io import BytesIO
from matplotlib.pyplot import imread
from scipy.ndimage.interpolation import zoom, rotate
import cv2
from tqdm import tqdm
import pandas as pd



# Device configuration, as before
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#Our try with ori#

#This wlll be a reference to the other 2 versions of the building of the dataset

the main dunction are:

1) init

2) getitem

3) data function such as "flip" "crop" "normalize" and so on

In [ ]:

class TrainDataset(Dataset):
    def __init__(self, imgs_path, targets_path):  # will have targets here?
        self.imgs_path = pd.read_csv(imgs_path)
        self.target_file = pd.read_csv(targets_path)

    def __len__(self):
        # return how many items in dataset
        # ...
        return

    def __getitem__(self, index):
        current_image = image.load(self.imgs_path + str(index) + '.png')
        current_target = self.target_file[index]

        ### Transforms:
        # resize
        # normalization (x - mean) / std
        # flip randomly  -- augmentation
        # random crop -- augmentation
        # to tensor: current_image = torch.tensor(current_image)  (torch.FloatTensor(current_image))

        return current_image, current_target


NUM_EPOCHS = 100
imgs_path = "/Ori/images/"   #  1000 images 0.png - 999.png
targets_path = "/Ori/targets.csv"   #  1000 images 0.png - 999.png

train_dataset = TrainDataset(imgs_path, targets_path)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for epoch in range(NUM_EPOCHS):
    # for i, (batch, target) in enumerate(train_dataloader):
    for i, batch in enumerate(train_dataloader):
        res = model(batch)
        loss = ((res - batch) ** 2).mean()  ## MSE

        if i % 10 == 0:
            evaluate_model()


FileNotFoundError: ignored

#GLRDataset

this dataset is based on the git of the third and first place of the retrivel competition of 2019,

so it's a pretty conplex dataset, so we are not going to use it, but we want to use and explore more of those functions:

**def__init__** = initialze the datset and define the function that we need

**_getitem_** = load one picture from the database and can use one of the data functions to activate on the returned data

**augment** = increase the size of the pis to a "spesific" size

**normalize_img** = normalize the pictures sizes or normalize the picture pixels  

In [ ]:
###------Our GLR dataset - first try 13/01------###

class GLRDataset(Dataset):

    def __init__(self, df, suffix='.jpg', preload=False, aug = None, normalization='simple'):

        self.df = df  
        self.aug = aug
        self.normalization = normalization
        self.labels = self.df.target.values
        self.img_folder = self.df.img_folder.values
        self.suffix = suffix
        self.image_names = self.df.id.values
        self.images_cache = {}
        self.images_in_cache = False

        if preload:
            self.preload()
            self.images_in_cache = True
        self.eps = 1e-6

    def __getitem__(self, idx):
        id_ = self.image_names[idx]
        img_folder_ = self.img_folder[idx]
        
        if self.images_in_cache:
            img = self.images_cache[id_]
        else:
            img = self.load_one(id_, img_folder_)
            
        if self.aug:
            img = self.augment(img)
                
        img = img.astype(np.float32)       
        if self.normalization:
            img = self.normalize_img(img)
    
        tensor = self.to_torch_tensor(img)
        
        target = torch.tensor(self.labels[idx])
        feature_dict = {'idx':torch.tensor(idx).long(),
                        'input':tensor,
                       'target':target.float()}
        return feature_dict

    def __len__(self):
        return len(self.image_names)


    def preload(self):
        if self.n_threads > 1:
            with mp.Pool(self.n_threads) as p:
                imgs = p.map(self.load_one,self.id)
            self.images_cache = dict(zip(self.id, imgs))
        else:
            for i in tqdm(self.id):
                self.images_cache[i] = self.load_one(i)

    def load_one(self, id_, img_folder_):
        try:
            img = cv2.imread(img_folder_ + f'{id_[0]}/{id_[1]}/{id_[2]}/{id_}{self.suffix}')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB )
        except:
            print("FAIL READING IMG", img_folder_ + f'{id_[0]}/{id_[1]}/{id_[2]}/{id_}{self.suffix}')
            img = np.zeros((512,512,3), dtype=np.int8)
        return img

    def augment(self,img):
        img_aug = self.aug(image=img)['image']
        return img_aug.astype(np.float32)

    def normalize_img(self,img):
        
        if self.normalization == 'channel':
            pixel_mean = img.mean((0,1))
            pixel_std = img.std((0,1)) + self.eps
            img = (img - pixel_mean[None,None,:]) / pixel_std[None,None,:]
            img = img.clip(-20,20)

        elif self.normalization == 'channel_mean':
            pixel_mean = img.mean((0,1))
            img = (img - pixel_mean[None,None,:])
            img = img.clip(-20,20)
            
        elif self.normalization == 'image':
            img = (img - img.mean()) / img.std() + self.eps
            img = img.clip(-20,20)
            
        elif self.normalization == 'simple':
            img = img/255
            
        elif self.normalization == 'inception':
            
            mean = np.array([0.5, 0.5 , 0.5], dtype=np.float32)
            std = np.array([0.5, 0.5 , 0.5], dtype=np.float32)
            img = img.astype(np.float32)
            img = img/255.
            img -= mean
            img *= np.reciprocal(std, dtype=np.float32)
            
        elif self.normalization == 'imagenet':
            
            mean = np.array([123.675, 116.28 , 103.53 ], dtype=np.float32)
            std = np.array([58.395   , 57.120, 57.375   ], dtype=np.float32)
            img = img.astype(np.float32)
            img -= mean
            img *= np.reciprocal(std, dtype=np.float32)
            
        else:
            pass
        
        return img
    
    
    def to_torch_tensor(self,img):
        return torch.from_numpy(img.transpose((2, 0, 1)))
    

In [ ]:
###---dosen't needthis functiom---###

# def grab(url):
#     response = request.urlopen(url)
#     image_data = response.read()

#     img = Image.open(BytesIO(image_data))

#     img = img.convert('RGB')

#     return np.array(img)


# def sample(dataframe):
#     img = None
#     while(img is None):
#         try:
#             img = grab(dataframe.sample(1).iloc[0, 1])
#         except:
#             img = None

#     return img

###---dosen't needthis functiom---###
def read_img(img_id):
    try: # Try reading the downloaded data first
        return imread('./test/' + img_id + '.jpg')
    except: # Otherwise, try grabbing it off the web
        try:
            return grab(test_dict[img_id])
        except: # OK, so the image doesn't exist, just return a None
            return None


def random_crop(img, size):
    idx = np.random.randint(0, img.shape[0] - size[0])
    idy = np.random.randint(0, img.shape[1] - size[1])

    return img[idx:idx + size[0], idy:idy + size[1]]


class RandomFlip(object):
    """Horizontally flip the given NumPy array randomly with a given probability.
    Args:
        axis (positive integer): axis to flip along
        p (float): probability of the image being flipped. Default value is 0.5
    """

    def __init__(self, axis = 0, p = 0.5):
        self.axis = axis
        self.p = p

    def __call__(self, x):
        """
        Args:
            x (NumPy array): array to be flipped.
        Returns:
            NumPy array: Randomly flipped NumPy array.
        """
        if random.random() < self.p:
            return np.flip(x, axis = self.axis).copy()
        return x


class RandomRotation(object):
    """Rotate the image by angle.
    Args:
        degrees (min, max): Range of degrees to select from.
    """

    def __init__(self, degrees):
        self.degrees = degrees

    def __call__(self, img):
        """
            img (PIL Image): Image to be rotated.
        Returns:
            PIL Image: Rotated image.
        """

        angle = random.uniform(self.degrees[0], self.degrees[1])

        return rotate(img, angle, order = 1, reshape = False)


class CSVDataset(Dataset): # Note: All torchvision transforms are just classes with a __call__ attribute anyway, so they can be used here
    def __init__(self, dataframe, directory, transforms = None, submission = False):
        self.directory = directory
        self.dataframe = dataframe #didn't fully understand the data frame of the picture
        self.submission = submission ##the submission of the projest 
        self.transforms = transforms ##the type of tansformation 

    def __getitem__(self, index):
        row = self.dataframe.iloc[index] ## the 
        url = row['url']
        idx = row['id']
        if self.submission:
            category = torch.LongTensor([-1])[0]
        else:
            category = torch.LongTensor([row['landmark_id']])[0]

        img = imread(self.directory + idx + '.jpg')
        #img = zoom(img, (224.0 / img.shape[0], 224.0 / img.shape[1], 1), order = 1)
        #img = Image.open(self.directory + idx + '.jpg')

        img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_LINEAR)
        if self.transforms is not None:
            img = self.transforms(img)
        img = torch.from_numpy(np.transpose(img, (2, 0, 1)))

        if self.submission:
            return img, idx
        return img, category

    def __len__(self):
        return len(self.dataframe)

In [17]:
#all_data = pd.read_csv('/home/data/LandmarkRetrieval/train_clean.csv') # This has just under 100k images
#train_data, val_data = split_validation(all_data, 0.8)
train_data = pd.read_csv('/home/data/LandmarkRetrieval/train_split.csv') ## Ask ori how to import the csv files
val_data = pd.read_csv('/home/data/LandmarkRetrieval/val_split.csv') ## Ask ori how to import the csv files


# classes = int(max(np.max(val_data['landmark_id']), np.max(train_data['landmark_id']))) + 1

print('Training samples: ', len(train_data), '\n', train_data.head())
print('Validation samples: ', len(val_data), '\n', val_data.head())


transforms = Compose([RandomFlip(axis = 0), RandomFlip(axis = 1), RandomRotation(degrees = (-20, 20))])

train_set = CSVDataset(train_data, '/home/data/LandmarkRetrieval/train/', transforms = transforms)
val_set = CSVDataset(val_data, '/home/data/LandmarkRetrieval/train/')

# 16 seems to be the maximum batchsize we can do parallel load and train with
train_loader = DataLoader(train_set, batch_size = 16, sampler = sampler, num_workers = 6, pin_memory = True)
val_loader = DataLoader(val_set, batch_size = 16, shuffle = True, num_workers = 6, pin_memory = True)


FileNotFoundError: ignored

In [ ]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
train_dataset = datasets.CIFAR10(root='data', train=True,
                                   download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
